<a href="https://colab.research.google.com/github/Shreyash007/deep_learning_course/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 1 CS6910**
Shreyash Gadgil (ED22S016)

In [1]:
!pip install --upgrade wandb
!wandb login aa5afea12b4fda1e7f8310b597eb17c73d1176d2 #my API key for wandb login 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 12.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=a82098d41725b9590672836d124f270ca42975042f5f6f957bdb29da48b35fe8
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
wandb: A

Import libraries

In [2]:
from keras.datasets import fashion_mnist
import numpy as np
import math
import matplotlib.pyplot as plt
import wandb

Q1 SOLUTION
Labelling dataset

In [10]:
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()

#checking size of training and test dataset
print("Training dataset shape, X=",X_train.shape,", Y=",Y_train.shape)
print("Test dataset shape, X=",X_test.shape,", Y=",Y_test.shape)

n_img_classes=len(np.unique(Y_train)) #to find out number of unique images 
img_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] #taken from keras documentation


def show_images(n_img_classes,img_labels,X,Y):

   setImgs = []
   setLabels=[]

   for i in range(len(X)):
     if len(setImgs)==n_img_classes:
       break
     if img_labels[Y[i]] not in setLabels:
      setImgs.append(X[i])
      setLabels.append(img_labels[Y[i]])

   run=wandb.init(project='CS-6910 A1',entity='shreyashgadgil007',reinit=True)
   run.log({"Fashion_MNIST dataset":list(wandb.Image(img,caption= caption) for img,caption in zip(setImgs,setLabels))})
   run.finish()

#----Q1 SOLUTION
show_images(n_img_classes,img_labels,X_test,Y_test)


Training dataset shape, X= (60000, 28, 28) , Y= (60000,)
Test dataset shape, X= (10000, 28, 28) , Y= (10000,)


Feedforward neural network class

In [ ]:
def relu(X):
  return max(X,0)

def grad_relu(X):
  return X>0

def sigmoid(X):
  return 1/(1+np.exp(-X))

def grad_sigmoid(X):
  return sigmoid(X)(1-sigmoid(X))

def tanh(X):
  return (np.exp(X)-np.exp(-X))/(np.exp(X)+np.exp(-X))

def grad_tanh(X):
  return 1-(tanh(X))**2

def softmax(X):
  # Calculates the softmax function
  e_X = np.exp(X - np.max(X, axis = 0))
  return e_X / e_X.sum(axis = 0)

def softmax_cross_entropy(Y,Y_pred):
  return -(Y - Y_pred) 



#----INITIALISATION
def random_initialisation(shape):
  # Initialising a random matrix with given dimensions (shape) as tuple
  np.random.seed(0)
  return np.random.randn(*shape)*0.1

#return the values
def xavier_initialisation(shape):
 return